In [17]:
import joblib, os, librosa
from pydub import AudioSegment
import pandas as pd
from tqdm import tqdm
import numpy as np

In [18]:
laughter_classifier = joblib.load('../data/02_laughter/laughter_classifier_ model.pkl')
laughter_classifier

RandomForestClassifier()

In [19]:
def chop_and_execute(mp3_file, segment_duration=1.49, my_classification_model = 'insert_mode', command="echo 'Processing temp.mp3'"):
    """
    Chops an MP3 file into segments and executes a command on each segment.

    Args:
        mp3_file (str): Path to the input MP3 file.
        segment_duration (int): Duration of each segment in seconds.
        command (str): Command to execute on each segment (e.g., "python my_script.py").
    """

    try:
        audio = AudioSegment.from_mp3(mp3_file)
        total_duration_ms = len(audio)
        segment_duration_ms = segment_duration * 1000

        onsets           = []
        offsets          = []
        laughter_dummies = []

        for start_ms in tqdm(range(0, int(total_duration_ms), int(segment_duration_ms))):
            
            end_ms = min(start_ms + segment_duration_ms, total_duration_ms)
            segment = audio[start_ms:end_ms]

            temp_file = "../data/02_laughter/temp/" + str(start_ms) + ".mp3"
            segment.export(temp_file, format="mp3")

            #print(f"Processing segment {start_ms/1000}-{end_ms/1000}s")
            onsets.append(start_ms)
            offsets.append(end_ms)
            
            yes_no_laughter = detect_laughter(my_classification_model, temp_file)
            laughter_dummies.append(yes_no_laughter)
            os.remove(temp_file)
            

        df            = pd.DataFrame()
        df['onsets']  = onsets
        df['minutes_seconds'] = seconds_to_minutes_seconds(df['onsets'])
        df['offsets'] = offsets
        df['ls']      = np.asarray(laughter_dummies)
        
        
    except FileNotFoundError:
        print(f"Error: MP3 file not found: {mp3_file}")

    return df

def detect_laughter(model, audio_file, threshold=0.4):
    # Extract features
    features = extract_features(audio_file)
    
    # Reshape features to 2D array with shape [1, n_features]
    features = features.reshape(1, -1)  # Add this line
    
    # Predict probability
    laughter_prob = model.predict_proba(features)[:, 1]
    
    # Apply threshold
    laughter_segments = (laughter_prob > threshold).astype(int)

    
    
    return laughter_segments

def extract_features(audio_file, frame_length=0.1, hop_length=0.05):
    """Extract features from audio file"""
    # Load audio
    y, sr = librosa.load(audio_file, sr=None)
    
    # Set frame and hop length in samples
    frame_len_samples = int(frame_length * sr)
    hop_len_samples = int(hop_length * sr)
    
    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, 
                                hop_length=hop_len_samples, 
                                n_fft=frame_len_samples)
    
    # Extract spectral features
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr, 
                                                        hop_length=hop_len_samples,
                                                        n_fft=frame_len_samples)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr,
                                                          hop_length=hop_len_samples,
                                                          n_fft=frame_len_samples)
    
    # Extract temporal features
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y, 
                                                         frame_length=frame_len_samples, 
                                                         hop_length=hop_len_samples)
    
    # Extract energy
    energy = np.array([sum(abs(y[i:i+frame_len_samples]**2)) 
                      for i in range(0, len(y), hop_len_samples)])
    
    # Calculate frame-level features
    mfcc_means = np.mean(mfccs, axis=1)
    mfcc_stds = np.std(mfccs, axis=1)
    
    # Reshape single-value features to make them consistent
    # The issue is that spectral_centroid, spectral_bandwidth, etc. are 1D arrays
    # but we need them to be the same shape as the MFCC features for stacking
    
    # Get statistics as individual values instead of arrays
    spectral_centroid_mean = np.mean(spectral_centroid)
    spectral_bandwidth_mean = np.mean(spectral_bandwidth)
    zcr_mean = np.mean(zero_crossing_rate)
    zcr_std = np.std(zero_crossing_rate)
    energy_mean = np.mean(energy)
    energy_std = np.std(energy)
    
    # Combine all features into a single vector
    feature_vector = np.hstack([
        mfcc_means,
        mfcc_stds,
        spectral_centroid_mean,
        spectral_bandwidth_mean,
        zcr_mean,
        zcr_std,
        energy_mean,
        energy_std
    ])
    
    return feature_vector

def seconds_to_minutes_seconds(seconds_series):
    """
    Converts a pandas Series of seconds into a Series of [minutes:seconds] strings.
    Args: seconds_series (pd.Series): A pandas Series containing seconds (numeric).
    Returns:  pd.Series: A pandas Series of strings in the format [minutes:seconds].
    """
    seconds_series = seconds_series//1000
    minutes = seconds_series // 60
    remaining_seconds = seconds_series % 60
    return minutes.astype(str) + ':' + remaining_seconds.astype(str).str.zfill(2)

import os

def check_file_exists(filepath):
    """
    Check if a file exists at the specified path.
    
    Parameters:
    -----------
    filepath : str
        Path to the file to check
        
    Returns:
    --------
    bool
        True if the file exists, False otherwise
    """
    return os.path.isfile(filepath)





In [20]:
#audio_file_to_process = '/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e01a.mp3'
#df = chop_and_execute(audio_file_to_process, segment_duration=1.49, my_classification_model = laughter_classifier)

### Run over all files

In [21]:

folder_path = '/Volumes/T7 Shield/friends/stimuli/'  # <-- change to your folder
mp3_files = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.lower().endswith('.mp3') and "._" not in file:
            mp3_files.append(os.path.join(root, file))


mp3_files = remove_dot_underscore_files(mp3_files)
mp3_files

['/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e13a.mp3',
 '/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e09b.mp3',
 '/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e05a.mp3',
 '/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e13b.mp3',
 '/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e09a.mp3',
 '/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e05b.mp3',
 '/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e24a.mp3',
 '/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e04b.mp3',
 '/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e08a.mp3',
 '/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e12b.mp3',
 '/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e24b.mp3',
 '/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e04a.mp3',
 '/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e08b.mp3',
 '/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e12a.mp3',
 '/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e19a.mp3',
 '/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e15b.mp3',
 '/Volum

In [22]:
# print result
for f in mp3_files:
    print(f[38:53])
    out_name = '../data/02_laughter/laughter_annotations/' + f[38:53] + '.csv'

    if check_file_exists(out_name):
        print(f"The file '{out_name}' exists. Skipping. ")
    else:

        df = chop_and_execute(f, segment_duration=1.49, my_classification_model = laughter_classifier)
        df.to_csv(out_name)
    


friends_s01e13a
The file '../data/02_laughter/laughter_annotations/friends_s01e13a.csv' exists. Skipping. 
friends_s01e09b
The file '../data/02_laughter/laughter_annotations/friends_s01e09b.csv' exists. Skipping. 
friends_s01e05a
The file '../data/02_laughter/laughter_annotations/friends_s01e05a.csv' exists. Skipping. 
friends_s01e13b
The file '../data/02_laughter/laughter_annotations/friends_s01e13b.csv' exists. Skipping. 
friends_s01e09a
The file '../data/02_laughter/laughter_annotations/friends_s01e09a.csv' exists. Skipping. 
friends_s01e05b
The file '../data/02_laughter/laughter_annotations/friends_s01e05b.csv' exists. Skipping. 
friends_s01e24a
The file '../data/02_laughter/laughter_annotations/friends_s01e24a.csv' exists. Skipping. 
friends_s01e04b
The file '../data/02_laughter/laughter_annotations/friends_s01e04b.csv' exists. Skipping. 
friends_s01e08a
The file '../data/02_laughter/laughter_annotations/friends_s01e08a.csv' exists. Skipping. 
friends_s01e12b
The file '../data/02_

100%|█████████████████████████████████████████| 488/488 [02:14<00:00,  3.64it/s]


friends_s02e07b


100%|█████████████████████████████████████████| 496/496 [01:44<00:00,  4.73it/s]


friends_s02e11a


...

100%|█████████████████████████████████████████| 453/453 [01:34<00:00,  4.78it/s]


friends_s06e22b


100%|█████████████████████████████████████████| 486/486 [01:41<00:00,  4.77it/s]


friends_s06e15d


100%|█████████████████████████████████████████| 490/490 [01:42<00:00,  4.79it/s]
